In [1]:
import numpy as np
import ctypes
import pygame
from pygame.locals import DOUBLEBUF, OPENGL
from OpenGL.GL import *
from OpenGL.GLU import *
import math
import time
import random

# ---- Simplified Particle System ----
class ParticleSystem:
    def __init__(self, center=(0, 0, 0), size=2.0, particle_count=35):
        self.center = center
        self.size = size
        self.particles = []
        self.initialize_particles(particle_count)
    
    def initialize_particles(self, particle_count):
        """Create a group of independent particles arranged in a cube shape."""
        self.particles = []
        
        # Determine grid dimensions - aim for roughly cubic arrangement
        n = int(round(particle_count ** (1/3)))
        if n < 2:
            n = 2
            
        # Calculate step size
        step = self.size / (n - 1) if n > 1 else self.size
        
        # Generate particles in a grid
        for i in range(n):
            for j in range(n):
                for k in range(n):
                    if len(self.particles) >= particle_count:
                        break
                        
                    # Calculate position
                    x = self.center[0] - self.size/2 + i * step
                    y = self.center[1] - self.size/2 + j * step
                    z = self.center[2] - self.size/2 + k * step
                    
                    # Add variation for more natural look
                    x += random.uniform(-0.05, 0.05)
                    y += random.uniform(-0.05, 0.05)
                    z += random.uniform(-0.05, 0.05)
                    
                    # Add particle
                    self.particles.append({
                        'position': [x, y, z],
                        'velocity': [0, 0, 0],
                        'radius': random.uniform(0.1, 0.25),  # Random size
                        'mass': random.uniform(0.01, 0.05),     # Random mass
                        'ambient': [0.2, 0.1, 0.3, 1.0],      # Purple ambient
                        'diffuse': [0.4, 0.2, 0.6, 1.0],      # Purple diffuse
                        'specular': [0.6, 0.6, 0.7, 1.0],     # Light specular
                        'shininess': 30.0,                    # Medium shininess
                        'collision_time': 0,                  # To track collision effects
                        'on_ground': False
                    })
    
    def update(self, dt, ball_position, ball_radius, current_time):
        """Update all particles with physics."""
        collision_occurred = False
        
        # Apply physics to each particle
        for i, particle in enumerate(self.particles):
            # Apply gravity
            # particle['velocity'][1] -= 9.81 * dt
            particle['velocity'][1] -= 35 * dt
            
            # Apply damping (air resistance)
            for j in range(3):
                particle['velocity'][j] *= 0.981  # Slightly reduced damping for more movement
            
            # Update position
            for j in range(3):
                particle['position'][j] += particle['velocity'][j] * dt
            
            # Wall collisions
            wall_size = 5.0
            
            # Floor collision
            if particle['position'][1] - particle['radius'] < -1.0:
                particle['position'][1] = -1.0 + particle['radius']
                if particle['velocity'][1] < 0:
                    # Stronger bounce for more noticeable floor interaction
                    particle['velocity'][1] = -particle['velocity'][1] * 0.99  
                    
                    # Apply friction for particles on ground
                    particle['velocity'][0] *= 0.99
                    particle['velocity'][2] *= 0.99
                
                # Mark as on ground
                particle['on_ground'] = True
                
                # Change color for particles on ground (reddish)
                #particle['ambient'] = [0.2, 0.1, 0.1, 1.0]
                #particle['diffuse'] = [0.7, 0.3, 0.3, 1.0]
            else:
                particle['on_ground'] = False
            
            # Ceiling collision
            if particle['position'][1] + particle['radius'] > 5.0:
                particle['position'][1] = 5.0 - particle['radius']
                if particle['velocity'][1] > 0:
                    particle['velocity'][1] = -particle['velocity'][1] * 0.65
            
            # X walls
            if particle['position'][0] + particle['radius'] > wall_size:
                particle['position'][0] = wall_size - particle['radius']
                if particle['velocity'][0] > 0:
                    particle['velocity'][0] = -particle['velocity'][0] * 0.65
            elif particle['position'][0] - particle['radius'] < -wall_size:
                particle['position'][0] = -wall_size + particle['radius']
                if particle['velocity'][0] < 0:
                    particle['velocity'][0] = -particle['velocity'][0] * 0.65
            
            # Z walls
            if particle['position'][2] + particle['radius'] > wall_size:
                particle['position'][2] = wall_size - particle['radius']
                if particle['velocity'][2] > 0:
                    particle['velocity'][2] = -particle['velocity'][2] * 0.65
            elif particle['position'][2] - particle['radius'] < -wall_size:
                particle['position'][2] = -wall_size + particle['radius']
                if particle['velocity'][2] < 0:
                    particle['velocity'][2] = -particle['velocity'][2] * 0.65
            
            # Ball collision
            dx = particle['position'][0] - ball_position[0]
            dy = particle['position'][1] - ball_position[1]
            dz = particle['position'][2] - ball_position[2]
            
            distance = math.sqrt(dx*dx + dy*dy + dz*dz)
            min_distance = particle['radius'] + ball_radius
            
            if distance < min_distance:
                # Normalize collision vector
                if distance > 0:
                    nx = dx / distance
                    ny = dy / distance
                    nz = dz / distance
                else:
                    # Avoid division by zero with random direction
                    nx, ny, nz = random.uniform(-1, 1), random.uniform(0, 1), random.uniform(-1, 1)
                    norm = math.sqrt(nx*nx + ny*ny + nz*nz)
                    nx, ny, nz = nx/norm, ny/norm, nz/norm
                
                # Move out of collision
                overlap = min_distance - distance
                particle['position'][0] += nx * overlap * 1.01  # Slight extra push to avoid sticking
                particle['position'][1] += ny * overlap * 1.01
                particle['position'][2] += nz * overlap * 1.01
                
                # Relative velocity along normal
                vx = particle['velocity'][0] - 0  # Assuming ball doesn't move for simplicity
                vy = particle['velocity'][1] - 0
                vz = particle['velocity'][2] - 0
                
                vn = vx*nx + vy*ny + vz*nz
                
                # Only bounce if moving toward each other
                if vn < 0:
                    # Impulse - assume ball has much higher mass
                    impulse = -(1.0 + 0.8) * vn * 1.5  # Increased bounce factor for more dramatic effect
                    
                    # Apply impulse to particle velocity
                    particle['velocity'][0] += impulse * nx
                    particle['velocity'][1] += impulse * ny
                    particle['velocity'][2] += impulse * nz
                    
                    # Add some randomization for more natural behavior
                    particle['velocity'][0] += random.uniform(-0.2, 0.2)
                    particle['velocity'][1] += random.uniform(0.1, 0.4)  # More upward bias
                    particle['velocity'][2] += random.uniform(-0.2, 0.2)
                
                # Mark collision
                collision_occurred = True
                
                # Change color and track collision time
                particle['ambient'] = [0.3, 0.2, 0.1, 1.0]  # Amber ambient
                particle['diffuse'] = [0.9, 0.6, 0.1, 1.0]  # Orange/gold diffuse
                particle['collision_time'] = current_time
        
        # Inter-particle collisions - with enhanced response
        for i in range(len(self.particles)):
            for j in range(i+1, len(self.particles)):
                p1 = self.particles[i]
                p2 = self.particles[j]
                
                # Vector from p1 to p2
                dx = p2['position'][0] - p1['position'][0]
                dy = p2['position'][1] - p1['position'][1]
                dz = p2['position'][2] - p1['position'][2]
                
                distance = math.sqrt(dx*dx + dy*dy + dz*dz)
                min_distance = p1['radius'] + p2['radius']
                
                # Check for collision
                if distance < min_distance:
                    # Record collision
                    collision_occurred = True
                    
                    # Normalize collision vector
                    if distance > 0:
                        nx = dx / distance
                        ny = dy / distance
                        nz = dz / distance
                    else:
                        # Avoid division by zero
                        nx, ny, nz = 1, 0, 0
                    
                    # Move particles apart - more aggressively to prevent sticking
                    overlap = min_distance - distance
                    
                    # Distribute displacement based on mass
                    total_mass = p1['mass'] + p2['mass']
                    p1_ratio = p2['mass'] / total_mass
                    p2_ratio = p1['mass'] / total_mass
                    
                    # More aggressive separation (1.1 factor) to prevent clumping
                    p1['position'][0] -= nx * overlap * p1_ratio * 1.1
                    p1['position'][1] -= ny * overlap * p1_ratio * 1.1
                    p1['position'][2] -= nz * overlap * p1_ratio * 1.1
                    
                    p2['position'][0] += nx * overlap * p2_ratio * 1.1
                    p2['position'][1] += ny * overlap * p2_ratio * 1.1
                    p2['position'][2] += nz * overlap * p2_ratio * 1.1
                    
                    # Calculate relative velocity
                    vx = p2['velocity'][0] - p1['velocity'][0]
                    vy = p2['velocity'][1] - p1['velocity'][1]
                    vz = p2['velocity'][2] - p1['velocity'][2]
                    
                    # Relative velocity along normal
                    vn = vx*nx + vy*ny + vz*nz
                    
                    # Only bounce if moving toward each other
                    if vn < 0:
                        # Calculate impulse (with enhanced elasticity)
                        restitution = 0.65  # Higher coefficient of restitution for bouncier collisions
                        impulse = -(1.0 + restitution) * vn / total_mass * 1.5  # Enhanced for more visible effect
                        
                        # Apply impulse to velocities
                        p1['velocity'][0] -= impulse * p2['mass'] * nx
                        p1['velocity'][1] -= impulse * p2['mass'] * ny
                        p1['velocity'][2] -= impulse * p2['mass'] * nz
                        
                        p2['velocity'][0] += impulse * p1['mass'] * nx
                        p2['velocity'][1] += impulse * p1['mass'] * ny
                        p2['velocity'][2] += impulse * p1['mass'] * nz
                        
                        # Slight color change to indicate collision
                        # Slightly adjust color based on collision - subtle blue tint
                        if not p1['on_ground']:
                            p1['diffuse'][0] *= 0.98
                            p1['diffuse'][2] *= 1.02
                            
                        if not p2['on_ground']:
                            p2['diffuse'][0] *= 0.98
                            p2['diffuse'][2] *= 1.02

                    # Occasionally add small random forces to keep particles active
                    if random.random() < 0.01:  # 1% chance per frame per particle
                        particle['velocity'][0] += random.uniform(-0.3, 0.3)
                        particle['velocity'][1] += random.uniform(0, 0.4)
                        particle['velocity'][2] += random.uniform(-0.3, 0.3)
        
        
        return collision_occurred
    
    def render(self):
        """Render all particles with lighting effects."""
        # Ensure lighting is enabled
        glEnable(GL_LIGHTING)
        
        # Create a quadric for spheres
        quadric = gluNewQuadric()
        
        for particle in self.particles:
            glPushMatrix()
            glTranslatef(
                particle['position'][0],
                particle['position'][1],
                particle['position'][2]
            )
            
            # Set material properties for lighting
            glMaterialfv(GL_FRONT, GL_AMBIENT, particle['ambient'])
            glMaterialfv(GL_FRONT, GL_DIFFUSE, particle['diffuse'])
            glMaterialfv(GL_FRONT, GL_SPECULAR, particle['specular'])
            glMaterialf(GL_FRONT, GL_SHININESS, particle['shininess'])
            
            # Draw as sphere - more detail for cleaner look
            radius = particle['radius']
            # Use more detail for larger particles
            slices = 12 if radius > 0.12 else 8
            stacks = slices
            gluSphere(quadric, radius, slices, stacks)
            
            glPopMatrix()
        
        # Clean up
        gluDeleteQuadric(quadric)

# ---- Interactive Physics Class ----
class InteractivePhysics:
    """Python wrapper for interactive CUDA Fortran physics."""
    
    def __init__(self, lib_path='./one_particle5.so'):
        try:
            self.lib = ctypes.CDLL(lib_path)
            print(f"Successfully loaded library: {lib_path}")
            
            # Set up function signatures
            self.lib.init_physics.argtypes = []
            self.lib.init_physics.restype = None
            
            self.lib.cleanup_physics.argtypes = []
            self.lib.cleanup_physics.restype = None
            
            self.lib.reset_physics.argtypes = []
            self.lib.reset_physics.restype = None
            
            self.lib.update_physics.argtypes = [ctypes.c_double]  # timestep
            self.lib.update_physics.restype = None
            
            self.lib.get_particle_position.argtypes = [
                ctypes.POINTER(ctypes.c_double),
                ctypes.POINTER(ctypes.c_double),
                ctypes.POINTER(ctypes.c_double)
            ]
            self.lib.get_particle_position.restype = None
            
            # Set up shooting function signature
            self.lib.shoot_particle.argtypes = [
                ctypes.c_double,  # dir_x
                ctypes.c_double,  # dir_y
                ctypes.c_double,  # dir_z
                ctypes.c_double   # force_magnitude
            ]
            self.lib.shoot_particle.restype = None
            
            # Initialize physics
            self.lib.init_physics()
            
            # Position array
            self.position = np.zeros(3, dtype=np.float64)
            self.prev_position = np.zeros(3, dtype=np.float64)
            
            print("Physics system initialized")
            self.initialized = True
            
        except Exception as e:
            print(f"Error initializing physics: {e}")
            self.initialized = False
    
    def update(self, dt):
        """Update the physics by one timestep."""
        if not self.initialized:
            return
        
        try:
            # Save previous position to calculate velocity
            self.prev_position = np.copy(self.position)
            
            # Update physics
            self.lib.update_physics(ctypes.c_double(dt))
            
            # Get updated particle position
            pos_x = ctypes.c_double()
            pos_y = ctypes.c_double()
            pos_z = ctypes.c_double()
            
            self.lib.get_particle_position(
                ctypes.byref(pos_x),
                ctypes.byref(pos_y),
                ctypes.byref(pos_z)
            )
            
            # Update position array
            self.position[0] = pos_x.value
            self.position[1] = pos_y.value
            self.position[2] = pos_z.value
            
            # Apply wall constraints (if not handled in Fortran)
            # We now handle these in the Fortran module, but keeping this as a safety
            wall_size = 5.0  # Half the width of the walls (wall goes from -wall_size to +wall_size)
            
            if abs(self.position[0]) > wall_size:
                sign = 1 if self.position[0] > 0 else -1
                self.position[0] = sign * wall_size
                
            if self.position[1] < -1.0:  # Floor
                self.position[1] = -1.0
                
            if self.position[1] > wall_size:  # Ceiling
                self.position[1] = wall_size
                
            if abs(self.position[2]) > wall_size:
                sign = 1 if self.position[2] > 0 else -1
                self.position[2] = sign * wall_size
                
        except Exception as e:
            print(f"Error in update: {e}")
    
    def get_velocity(self, dt):
        """Calculate velocity based on position change."""
        if dt <= 0:
            return np.zeros(3)
        
        return (self.position - self.prev_position) / dt
    
    def shoot(self, dir_x, dir_y, dir_z, power):
        """Shoot the particle in the specified direction with the given power."""
        if not self.initialized:
            return
            
        try:
            # Direction inversion to match coordinate system
            self.lib.shoot_particle(
                ctypes.c_double(-dir_x),
                ctypes.c_double(-dir_y),
                ctypes.c_double(-dir_z),
                ctypes.c_double(power)
            )
            print(f"Shot particle: dir=({-dir_x:.2f}, {-dir_y:.2f}, {-dir_z:.2f}), power={power:.2f}")
        except Exception as e:
            print(f"Error in shoot: {e}")
    
    def reset(self):
        """Reset the physics to initial state."""
        if self.initialized:
            try:
                self.lib.reset_physics()
                print("Physics reset to initial state")
                # Reset position arrays
                self.position = np.zeros(3, dtype=np.float64)
                self.prev_position = np.zeros(3, dtype=np.float64)
            except Exception as e:
                print(f"Error in reset: {e}")
    
    def cleanup(self):
        """Clean up physics resources."""
        if self.initialized:
            try:
                self.lib.cleanup_physics()
                print("Physics resources cleaned up")
            except Exception as e:
                print(f"Error in cleanup: {e}")

# ---- Utility Functions ----
def draw_grid(size, step):
    """Draw a reference grid on the XZ plane."""
    glDisable(GL_LIGHTING)  # Disable lighting for grid
    
    glBegin(GL_LINES)
    
    # Main grid
    glColor4f(0.2, 0.2, 0.2, 0.5)  # Dark gray with transparency
    
    # Note: size is now an integer, so range() works correctly
    for i in range(-size, size+1, step):
        # Lines along X axis
        glVertex3f(-size, -1.0, i)
        glVertex3f(size, -1.0, i)
        
        # Lines along Z axis
        glVertex3f(i, -1.0, -size)
        glVertex3f(i, -1.0, size)
    
    glEnd()

def draw_glass_walls(size, height, alpha):
    """Draw transparent glass walls around the perimeter."""
    # Setup materials for glass effect
    glEnable(GL_LIGHTING)
    glEnable(GL_LIGHT0)
    
    # Back wall (Z = size) - Blue glass
    glMaterialfv(GL_FRONT_AND_BACK, GL_AMBIENT, (0.1, 0.1, 0.3, alpha))
    glMaterialfv(GL_FRONT_AND_BACK, GL_DIFFUSE, (0.2, 0.2, 0.8, alpha))
    glMaterialfv(GL_FRONT_AND_BACK, GL_SPECULAR, (1.0, 1.0, 1.0, alpha))
    glMaterialf(GL_FRONT_AND_BACK, GL_SHININESS, 100.0)
    
    glBegin(GL_QUADS)
    glColor4f(0.2, 0.2, 0.8, alpha)  # Stronger blue glass
    glNormal3f(0.0, 0.0, -1.0)  # Normal pointing inward
    glVertex3f(-size, -1.0, size)
    glVertex3f(size, -1.0, size)
    glVertex3f(size, height, size)
    glVertex3f(-size, height, size)
    glEnd()
    
    # Front wall (Z = -size) - Blue glass
    glMaterialfv(GL_FRONT_AND_BACK, GL_AMBIENT, (0.1, 0.1, 0.3, alpha))
    glMaterialfv(GL_FRONT_AND_BACK, GL_DIFFUSE, (0.2, 0.2, 0.8, alpha))
    
    glBegin(GL_QUADS)
    glColor4f(0.2, 0.2, 0.8, alpha)
    glNormal3f(0.0, 0.0, 1.0)
    glVertex3f(-size, -1.0, -size)
    glVertex3f(size, -1.0, -size)
    glVertex3f(size, height, -size)
    glVertex3f(-size, height, -size)
    glEnd()
    
    # Left wall (X = -size) - Red glass
    glMaterialfv(GL_FRONT_AND_BACK, GL_AMBIENT, (0.3, 0.1, 0.1, alpha))
    glMaterialfv(GL_FRONT_AND_BACK, GL_DIFFUSE, (0.8, 0.2, 0.2, alpha))
    
    glBegin(GL_QUADS)
    glColor4f(0.8, 0.2, 0.2, alpha)  # Stronger red glass
    glNormal3f(1.0, 0.0, 0.0)
    glVertex3f(-size, -1.0, -size)
    glVertex3f(-size, -1.0, size)
    glVertex3f(-size, height, size)
    glVertex3f(-size, height, -size)
    glEnd()
    
    # Right wall (X = size) - Red glass
    glMaterialfv(GL_FRONT_AND_BACK, GL_AMBIENT, (0.3, 0.1, 0.1, alpha))
    glMaterialfv(GL_FRONT_AND_BACK, GL_DIFFUSE, (0.8, 0.2, 0.2, alpha))
    
    glBegin(GL_QUADS)
    glColor4f(0.8, 0.2, 0.2, alpha)
    glNormal3f(-1.0, 0.0, 0.0)
    glVertex3f(size, -1.0, -size)
    glVertex3f(size, -1.0, size)
    glVertex3f(size, height, size)
    glVertex3f(size, height, -size)
    glEnd()
    
    # Ceiling (Y = height) - Gray glass
    glMaterialfv(GL_FRONT_AND_BACK, GL_AMBIENT, (0.2, 0.2, 0.2, alpha))
    glMaterialfv(GL_FRONT_AND_BACK, GL_DIFFUSE, (0.7, 0.7, 0.7, alpha))
    
    glBegin(GL_QUADS)
    glColor4f(0.7, 0.7, 0.7, alpha)  # Gray glass
    glNormal3f(0.0, -1.0, 0.0)
    glVertex3f(-size, height, -size)
    glVertex3f(size, height, -size)
    glVertex3f(size, height, size)
    glVertex3f(-size, height, size)
    glEnd()

# ---- Main Function ----
def main():
    """Main function for visualization."""
    # Initialize pygame
    pygame.init()
    display = (1920, 1080)  # Larger display for better visibility
    pygame.display.set_mode(display, DOUBLEBUF | OPENGL |pygame.FULLSCREEN)
    pygame.display.set_caption("Interactive Physics - Particle Simulation")

    frame_count = 0
    start_time = time.time()
    
    # Set up OpenGL
    glClearColor(0.05, 0.05, 0.1, 1.0)  # Dark blue background
    glEnable(GL_DEPTH_TEST)
    
    # Set up projection
    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    projection_matrix = glGetFloatv(GL_PROJECTION_MATRIX)
    gluPerspective(45, (display[0]/display[1]), 0.1, 50.0)
    projection_matrix = glGetFloatv(GL_PROJECTION_MATRIX)
    
    # Font for text display
    font = pygame.font.SysFont("Arial", 18)
    
    # Initialize physics
    physics = InteractivePhysics()
    if not physics.initialized:
        pygame.quit()
        return
    
    # Camera settings
    dist = 10.0
    theta = 45.0  # Angle around y-axis
    phi = 30.0    # Angle from xz-plane
    
    # Create a particle system
    particle_count = 100
    particles = ParticleSystem(center=(0, 1.5, 0), size=2.0, particle_count=particle_count)
    
    # Particle settings
    particle_radius = 0.2  # Should match the Fortran code
    particle_color = (0.9, 0.2, 0.2)  # Red
    
    # Create a surface for text overlay
    text_surface = pygame.Surface((400, 400), pygame.SRCALPHA)
    
    # Shooting controls
    shoot_direction = [0.5, 0.5, 0.0]  # Default direction (x, y, z)
    shoot_power = 3.0                  # Default power
    show_direction_line = False        # Option to toggle direction line - OFF by default
    
    # Create a surface for shooting controls
    controls_surface = pygame.Surface((300, 150), pygame.SRCALPHA)
    
    # Frame rate settings
    target_fps_options = [30, 60, 120, 165, 240]
    target_fps_index = 2  # Default to 60 FPS
    target_fps = target_fps_options[target_fps_index]
    actual_fps = 0.0
    
    # Simulation settings
    dt = 1.0 / target_fps  # Timestep based on target FPS
    physics_substeps = 2   # Number of physics steps per frame
    physics_dt = dt / physics_substeps  # Smaller dt for stable physics
    paused = False
    
    # Wall settings
    wall_size = 5.0  # Half the width of the walls
    wall_height = 5.0  # Height of the walls
    wall_alpha = 0.4  # Wall transparency
    
    # Lighting settings
    enable_lighting = True
    light_position = [0.0, wall_height - 1.0, 0.0, 1.0]  # Central ceiling light
    
    # Trail for visualization
    trail = []
    max_trail_length = 50
    show_trail = True
    
    # Add an initial impulse to start the ball moving
    initial_dir_x = 0.5
    initial_dir_z = 0.5
    initial_power = 4.0
    physics.shoot(initial_dir_x, 0.0, initial_dir_z, initial_power)
    print("Applied initial impulse to get things moving")
    
    # Main loop
    clock = pygame.time.Clock()
    running = True
    
    while running:
        current_time = time.time() - start_time
        modelview_matrix = None
        
        # Handle events
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    running = False
                elif event.key == pygame.K_SPACE:
                    paused = not paused
                elif event.key == pygame.K_r:
                    physics.reset()
                    trail = []  # Clear trail on reset
                    start_time = time.time()  # Reset time
                    # Recreate the particles
                    particles = ParticleSystem(center=(0, 1.5, 0), size=2.0, particle_count=particle_count)
                elif event.key == pygame.K_t:
                    show_trail = not show_trail
                elif event.key == pygame.K_l:
                    enable_lighting = not enable_lighting
                elif event.key == pygame.K_y:
                    show_direction_line = not show_direction_line
                # Frame rate controls
                elif event.key == pygame.K_f:
                    # Cycle through frame rate options
                    target_fps_index = (target_fps_index + 1) % len(target_fps_options)
                    target_fps = target_fps_options[target_fps_index]
                    dt = 1.0 / target_fps
                    physics_dt = dt / physics_substeps
                    print(f"Target FPS set to {target_fps}")
                # Shooting controls with keys
                elif event.key == pygame.K_s:
                    # Normalize direction vector
                    dir_length = math.sqrt(sum([d*d for d in shoot_direction]))
                    if dir_length > 0.0001:
                        dir_normalized = [d/dir_length for d in shoot_direction]
                        physics.shoot(
                            dir_normalized[0],
                            dir_normalized[1],
                            dir_normalized[2],
                            shoot_power
                        )
                # Direction adjustments
                elif event.key == pygame.K_1:
                    shoot_direction[0] = max(-1.0, shoot_direction[0] - 0.1)
                elif event.key == pygame.K_2:
                    shoot_direction[0] = min(1.0, shoot_direction[0] + 0.1)
                elif event.key == pygame.K_3:
                    shoot_direction[1] = max(-1.0, shoot_direction[1] - 0.1)
                elif event.key == pygame.K_4:
                    shoot_direction[1] = min(1.0, shoot_direction[1] + 0.1)
                elif event.key == pygame.K_5:
                    shoot_direction[2] = max(-1.0, shoot_direction[2] - 0.1)
                elif event.key == pygame.K_6:
                    shoot_direction[2] = min(1.0, shoot_direction[2] + 0.1)
                # Power adjustments
                elif event.key == pygame.K_MINUS:
                    shoot_power = max(1.0, shoot_power - 1.0)
                elif event.key == pygame.K_EQUALS:  # Plus key
                    shoot_power = min(20.0, shoot_power + 1.0)
                # Physics substeps adjustment
                elif event.key == pygame.K_LEFTBRACKET:  # [ key
                    physics_substeps = max(1, physics_substeps - 1)
                    physics_dt = dt / physics_substeps
                    print(f"Physics substeps: {physics_substeps}")
                elif event.key == pygame.K_RIGHTBRACKET:  # ] key
                    physics_substeps = min(10, physics_substeps + 1)
                    physics_dt = dt / physics_substeps
                    print(f"Physics substeps: {physics_substeps}")
                # Particle count controls
                elif event.key == pygame.K_MINUS or event.key == pygame.K_KP_MINUS:
                    particle_count = max(10, particle_count - 5)
                    print(f"Particle count: {particle_count}")
                    particles = ParticleSystem(center=(0, 1.5, 0), size=2.0, particle_count=particle_count)
                elif event.key == pygame.K_PLUS or event.key == pygame.K_KP_PLUS or event.key == pygame.K_EQUALS:
                    particle_count = min(100, particle_count + 5)
                    print(f"Particle count: {particle_count}")
                    particles = ParticleSystem(center=(0, 1.5, 0), size=2.0, particle_count=particle_count)
                    
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if event.button == 4:  # Scroll up
                    dist = max(2, dist - 0.5)
                elif event.button == 5:  # Scroll down:
                    dist = min(30, dist + 0.5)
                elif event.button == 1:  # Left click - shoot away from camera
                    # Get the current camera direction - this is more reliable
                    camera_forward_x = math.cos(math.radians(phi)) * math.cos(math.radians(theta))
                    camera_forward_y = math.sin(math.radians(phi))
                    camera_forward_z = math.cos(math.radians(phi)) * math.sin(math.radians(theta))
                    
                    # Use this direction directly - the negative means "away from camera"
                    physics.shoot(
                        camera_forward_x,  # Direction away from camera
                        camera_forward_y,
                        camera_forward_z,
                        shoot_power * 1.1  # Use higher power for more noticeable effect
                    )
                    print(f"Hit ball in direction: ({camera_forward_x:.2f}, {camera_forward_y:.2f}, {camera_forward_z:.2f})")
                        
            elif event.type == pygame.MOUSEMOTION:
                if pygame.mouse.get_pressed()[2]:  # Right button for camera
                    dx, dy = event.rel
                    theta += dx * 0.5
                    phi = max(-85, min(85, phi - dy * 0.5))
        
        # Update physics
        if not paused:
            # Multiple substeps for stability
            for _ in range(physics_substeps):
                physics.update(physics_dt)
            
            # Update trail
            if frame_count % 2 == 0 and show_trail:  # Add point every 2 frames
                # Make a copy of the position
                pos_copy = np.copy(physics.position)
                trail.append(pos_copy)
                
                # Keep trail at reasonable length
                if len(trail) > max_trail_length:
                    trail = trail[-max_trail_length:]
            
            # Update particles with smaller physics steps for stability
            particles_substeps = 2  # Can be different from ball physics
            for _ in range(particles_substeps):
                particles.update(dt / particles_substeps, physics.position, particle_radius, current_time)
        
        # Render 3D scene
        glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
        
        # Set up camera
        glMatrixMode(GL_MODELVIEW)
        glLoadIdentity()
        
        # Position camera using spherical coordinates
        eye_x = dist * math.cos(math.radians(phi)) * math.cos(math.radians(theta))
        eye_y = dist * math.sin(math.radians(phi))
        eye_z = dist * math.cos(math.radians(phi)) * math.sin(math.radians(theta))
        
        # Look at origin
        gluLookAt(eye_x, eye_y, eye_z, 0, 0, 0, 0, 1, 0)
        
        # Store the modelview matrix for ray casting
        modelview_matrix = glGetFloatv(GL_MODELVIEW_MATRIX)
        
        # Enable blending for transparency
        glEnable(GL_BLEND)
        glBlendFunc(GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA)
        
        # Set up lighting
        if enable_lighting:
            glEnable(GL_LIGHTING)
            glEnable(GL_LIGHT0)
            
            # Set light properties
            ambient = [0.2, 0.2, 0.2, 1.0]
            diffuse = [1.0, 1.0, 1.0, 1.0]
            specular = [1.0, 1.0, 1.0, 1.0]
            
            glLightfv(GL_LIGHT0, GL_POSITION, light_position)
            glLightfv(GL_LIGHT0, GL_AMBIENT, ambient)
            glLightfv(GL_LIGHT0, GL_DIFFUSE, diffuse)
            glLightfv(GL_LIGHT0, GL_SPECULAR, specular)
            
            # Global ambient light
            glLightModelfv(GL_LIGHT_MODEL_AMBIENT, [0.2, 0.2, 0.2, 1.0])
        
        # Draw reference grid
        draw_grid(int(wall_size), 1)
        
        # Disable depth mask for transparent objects
        glDepthMask(GL_FALSE)
        
        # Draw glass walls
        draw_glass_walls(wall_size, wall_height, wall_alpha)
        
        # Re-enable depth mask for opaque objects
        glDepthMask(GL_TRUE)
        
        # Disable lighting for trail
        glDisable(GL_LIGHTING)
        
        # Draw trail
        if show_trail and len(trail) > 1:
            glLineWidth(2.0)
            glBegin(GL_LINE_STRIP)
            
            for i, pos in enumerate(trail):
                # Fade the trail from red to transparent
                alpha = 0.8 * (i / len(trail))
                glColor4f(particle_color[0], particle_color[1], particle_color[2], alpha)
                glVertex3f(pos[0], pos[1], pos[2])
            
            glEnd()
            glLineWidth(1.0)
        
        # Draw the particles with lighting
        if enable_lighting:
            glEnable(GL_LIGHTING)
        particles.render()
        
        # Draw the ball with lighting
        if enable_lighting:
            glEnable(GL_LIGHTING)
            
            # Set material properties for the ball
            glMaterialfv(GL_FRONT, GL_AMBIENT, [0.2, 0.0, 0.0, 1.0])
            glMaterialfv(GL_FRONT, GL_DIFFUSE, [0.8, 0.0, 0.0, 1.0])
            glMaterialfv(GL_FRONT, GL_SPECULAR, [1.0, 1.0, 1.0, 1.0])
            glMaterialf(GL_FRONT, GL_SHININESS, 50.0)
        
        # Draw ball
        glPushMatrix()
        glTranslatef(physics.position[0], physics.position[1], physics.position[2])
        
        # Draw as sphere
        glColor3f(*particle_color)
        sphere = gluNewQuadric()
        gluSphere(sphere, particle_radius, 16, 16)
        gluDeleteQuadric(sphere)
        
        glPopMatrix()
        
        # Disable lighting for direction line
        glDisable(GL_LIGHTING)
        
        # Draw shooting direction if enabled (but off by default now)
        if show_direction_line:
            glPushMatrix()
            glTranslatef(physics.position[0], physics.position[1], physics.position[2])
            
            # Normalize and scale for visualization
            dir_length = math.sqrt(sum([d*d for d in shoot_direction]))
            if dir_length > 0.0001:
                dir_normalized = [d/dir_length for d in shoot_direction]
                scale = shoot_power * 0.2  # Scale based on power
                
                glLineWidth(3.0)
                glBegin(GL_LINES)
                glColor4f(1.0, 1.0, 0.0, 0.8)  # Yellow for direction
                glVertex3f(0, 0, 0)
                glVertex3f(
                    dir_normalized[0] * scale,
                    dir_normalized[1] * scale,
                    dir_normalized[2] * scale
                )
                glEnd()
                glLineWidth(1.0)
            
            glPopMatrix()
        
        # Render text overlay
        text_surface.fill((0, 0, 0, 128))  # Semi-transparent background
        
        # Get actual FPS (smoothed)
        actual_fps = 0.9 * actual_fps + 0.1 * clock.get_fps() if actual_fps > 0 else clock.get_fps()
        
        # Draw text info
        lines = [
            f"Particle position: ({physics.position[0]:.2f}, {physics.position[1]:.2f}, {physics.position[2]:.2f})",
            f"Camera: dist={dist:.1f}, theta={theta:.1f}°, phi={phi:.1f}°",
            f"{'PAUSED' if paused else 'RUNNING'}",
            f"Target FPS: {target_fps} (Press F to change)",
            f"Actual FPS: {actual_fps:.1f}",
            f"Physics steps per frame: {physics_substeps} (Press [ ] to adjust)",
            f"Particle count: {particle_count} (Press +/- to change)",
            f"Trail: {'ON' if show_trail else 'OFF'} (Press T to toggle)",
            f"Lighting: {'ON' if enable_lighting else 'OFF'} (Press L to toggle)",
            f"Direction Line: {'ON' if show_direction_line else 'OFF'} (Press Y to toggle)",
            "",
            "Controls:",
            "Space: Pause/Resume",
            "R: Reset Simulation",
            "T: Toggle Trail",
            "L: Toggle Lighting",
            "Y: Toggle Direction Line",
            "F: Cycle Frame Rate",
            "[ ]: Adjust Physics Steps",
            "S or Left Click: Shoot Particle",
            "1/2: Decrease/Increase X Direction",
            "3/4: Decrease/Increase Y Direction",
            "5/6: Decrease/Increase Z Direction",
            "-/=: Decrease/Increase Power",
            "+/-: Change Particle Count",
            "Right Mouse + Drag: Rotate Camera",
            "Scroll: Zoom In/Out",
            "ESC: Quit"
        ]
        
        y = 10
        for line in lines:
            text = font.render(line, True, (255, 255, 255))
            text_surface.blit(text, (10, y))
            y += 20
        
        # Draw shooting controls info
        controls_surface.fill((0, 0, 0, 160))  # Darker background
        
        # Display shooting parameters
        direction_str = f"Direction: ({shoot_direction[0]:.2f}, {shoot_direction[1]:.2f}, {shoot_direction[2]:.2f})"
        power_str = f"Power: {shoot_power:.1f}"
        
        dir_text = font.render(direction_str, True, (255, 255, 0))
        power_text = font.render(power_str, True, (255, 255, 0))
        shoot_text = font.render("Left Click or 'S' to SHOOT!", True, (255, 255, 255))
        
        controls_surface.blit(dir_text, (10, 10))
        controls_surface.blit(power_text, (10, 40))
        controls_surface.blit(shoot_text, (10, 70))
        
        # Blit surfaces to screen
        screen = pygame.display.get_surface()
        screen.blit(text_surface, (10, 10))
        screen.blit(controls_surface, (screen.get_width() - 310, 10))
        
        pygame.display.flip()
        
        # Limit to target FPS
        clock.tick(target_fps)
        frame_count += 1
    
    # Cleanup
    physics.cleanup()
    pygame.quit()

if __name__ == "__main__":
    main()

pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
Successfully loaded library: ./one_particle5.so
Physics system initialized
 Physics initialized with CUDA optimizations (with ceiling)
 Direction:   -0.5000000000000000        -0.000000000000000      
  -0.5000000000000000     
 Normalized:   -0.7071067811865475        -0.000000000000000      
  -0.7071067811865475     
 Impulse:    -7.071067811865475        -0.000000000000000      
   -7.071067811865475     
Shot particle: dir=(-0.50, -0.00, -0.50), power=4.00
Applied initial impulse to get things moving
 New velocity:    -7.071067811865475         0.000000000000000      
   -7.071067811865475     
 Direction:   -0.9329365494704148       -0.2672383760782568      
  -0.2412737967813583     
 Normalized: Shot particle: dir=(-0.93, -0.27, -0.24), power=3.30
Hit ball in direction: (0.93, 0.27, 0.24)
  -0.9329365494704149       -0.2672383760782569      
  -0.2412737967813583    